In [1]:
%load_ext beam_setup

# Use resource to access files and folders

A resource can be a file or folder path on: (1) local filesystem, s3 and s3 compatible filesystem, hdfs and sftp connection.

In [2]:
path = resource('/tmp/path/to/folder')

The path resource follows the pathlib api

In [3]:
path.mkdir()

In [4]:
file_path = path.joinpath('a.pt')

but it also supports read and write operations

In [5]:
file_path.write(torch.randn(4))

In [6]:
print(file_path.read())

it supports many file types including: torch (.pt), pickle (.pkl), feather (.fea), parquet (.parquet) and many more (see beam.path.core.PureBeamPath read and write operations)

we can also specify how we would like to store the file

In [7]:
path.joinpath('some_name_with_no_extention').write(np.random.randn(4), ext='.pkl')

In [8]:
path.joinpath('some_name_with_no_extention').read(ext='.pkl')

we can also iter and list folders

In [9]:
list(path.iterdir())
list(path)

To access paths on different storage platforms use the standard URI conventions:

path = resource('scheme://\<hostname\>:\<port\>/path/to/my/file?arg1=val1&arg2=val2')

Some examples:
* s3 on AWS: s3:///\<bucket name\>/<\object\>?access_key=\<my aws access key\>&secret_key=\<my secret access key\>
* s3 on Minio: s3://\<hostname\>:\<port\>/\<bucket name\>/<\object\>?access_key=\<my aws access key\>&secret_key=\<my secret access key\>?tls=false
* HDFS: hdfs://\<hostname\>:\<http connection port usually 9870\>/path/to/my/file?access_key=\<my hdfs access key\>?tls=\<whether connecting via https\>


Note that you can replace the scheme s3 with s3-pa and hdfs with hdfs-pa to get access via pyarrow which can increase performance instead of native implementations like boto3.
For hdfs-pa you may need to replace the port to the data node communication port: usually at 50010 (consult hdfs-site.xml and core-site.xml files for details)  

# Use resource to access Large Language Models (LLMs)

You can use resource also to access LLMs on various platforms: openai, fastchat, tgi, internal fastapi, local huggingface

before accessing to the LLM, note that we can store access keys to our environment and to a local file s.t. it stays permanenty in our system

In [10]:
# this would print the stored key in your system
# beam_key['OPENAI_API_KEY']

In [11]:
# this will assign a new key to your system
# beam_key['OPENAI_API_KEY'] = 'my_key_here'

In [12]:
llm = resource('openai:///gpt-4')

you can give any model instructions or chat with it

In [13]:
llm.ask('when israel was founded? give me exact date').text

you can chat with the model

In [14]:
llm.chat('Hi my name is elad').text

In [15]:
llm.chat('Hi again, do you remember my name?').text

In [16]:
llm.chat('Hi again, do you remember my name?', reset_chat=True).text

You can also parse the response in other formats

In [17]:
llm.ask('Hi how are you today? answer in a JSON format').json

In [18]:
llm.ask('Hi how are you today? answer in a YAML format').yaml

You can use the LLM also directly with langchain without any further wrapper

In [19]:
from langchain.schema import HumanMessage

text = "What would be a good company name for a company that makes colorful socks?"
messages = [HumanMessage(content=text)]

In [20]:
llm.invoke(text)

In [21]:
llm.invoke(messages)

With our URIs, You can also use openai syntax with any model (not just openai), simply import our simulator instead of openai

In [22]:
from beam.llm import openai_simulator as openai

In [23]:
llm = resource('openai:///gpt-4')

In [24]:
openai.Completion.create(prompt='2**10=?', model='openai:///text-davinci-003')

To access other LLM resource types follow the URI convention:

llm = resource('scheme://\<hostname\>:\<port\>/path/to/my/file?arg1=val1&arg2=val2')

possible schemes: openai, fastchat, tgi, fastapi, huggingface

# Use resource to access BeamServer algorithms

You can use beam also to quickly deploy an algorithm via ssh. You can then access this algorithm with a resource object from any machine that can access the server.

For example, here we build a sparse similarity server (like faiss but for sparse vectors e.g. TFIDF vectors)

In [26]:
from beam.sparse import SparseSimilarity
from beam.serve import beam_server
M = 40000

In [30]:
sparse_sim = SparseSimilarity(metric='cosine', format='coo', vec_size=M, device='cpu', k=10)
server = beam_server(sparse_sim, backend='waitress', non_blocking=True)

In [31]:
def gen_coo_vectors(k, M, nel):

    r = []
    c = []
    v = []

    for i in range(k):
        r.append(i * torch.ones(nel, dtype=torch.int64))
        c.append(torch.randint(M, size=(nel,)))
        v.append(torch.randn(nel))

    return torch.sparse_coo_tensor(torch.stack([torch.cat(r), torch.cat(c)]), torch.cat(v), size=(k, M))

In [32]:
s1 = gen_coo_vectors(20000, M, 100)
s2 = gen_coo_vectors(20, M, 100)

In [35]:
sparse_sim = resource('beam-http://localhost:27450')

In [36]:
sparse_sim.add(s1)

In [37]:
%%time
dist, ind = sparse_sim.search(s2, k=10)

The beam server can wrap any function or class for quick and easy deployment

In [38]:
def llm_executor_and_store(prompt, llm='openai:///text-davinci-003?max_tokens=2048'):
    llm = resource(llm)
    code = llm.ask(f"Return executable python code that performs the following task. The final result should assigned to a variable name 'res':\n{prompt}\n\n\n").text
    try:
        exec(code)
        return res, code
    except:
        return 'ExecutionError', code

In [39]:
beam_server(llm_executor_and_store, backend='waitress', non_blocking=True)

In [41]:
remote_executor = resource('beam-http://localhost:27451')

In [42]:
r, c = remote_executor('what is the 18th number in the fibonacci series?')

print(c)
print(r)